In [ ]:
# https://stable-baselines3.readthedocs.io/en/master/guide/rl.html
# https://spinningup.openai.com/en/latest/spinningup/rl_intro2.html#a-taxonomy-of-rl-algorithms

# 1. Import dependencies

In [1]:
!pip3 install stable-baselines3[extra]

In [3]:
pip install gymnasium

Note: you may need to restart the kernel to use updated packages.


In [62]:
pip install tensorboard

Note: you may need to restart the kernel to use updated packages.


In [2]:
import os
import gymnasium as gym
from stable_baselines3 import PPO
from stable_baselines3.common.vec_env import DummyVecEnv
from stable_baselines3.common.evaluation import evaluate_policy

# 2. Load Environment

In [3]:
environment_name = "CartPole-v1"

In [23]:
# Creates an environment previously registered with gymnasium.register() or a EnvSpec.
env = gym.make(environment_name)
#gym.make??

In [5]:
# Resets the environment to the initial state, required before calling step. 
# Returns the first agent observation for an episode and information, i.e. metrics, debug info.
env.reset()

(array([-0.03256245, -0.02750515, -0.04948962,  0.02764346], dtype=float32),
 {})

In [18]:
# This method defines how many discrete/actions there are. 
# The Space object corresponding to valid actions, all valid actions should be contained within the space.
env.action_space
# env.action_space.sample()

Discrete(2)

In [19]:
# This is the method that allows you to view the observation space
env.observation_space
# env.observation_space.sample()

Box([-4.8000002e+00 -3.4028235e+38 -4.1887903e-01 -3.4028235e+38], [4.8000002e+00 3.4028235e+38 4.1887903e-01 3.4028235e+38], (4,), float32)

### Run for loop of enivonrment

In [20]:
# Closes the environment, important when external software is used, i.e. pygame for rendering, databases
env.close()

In [10]:
env = gym.make(environment_name,render_mode="human")

In [11]:
episodes = 5
for episode in range(1, episodes+1):
    state = env.reset()
    done = False
    score = 0 
    
    while not done:
        env.render()
        action = env.action_space.sample()
        n_state, reward, done, info, __ = env.step(action)
        score+=reward
    print('Episode:{} Score:{}'.format(episode, score))
env.close()

Episode:1 Score:19.0
Episode:2 Score:39.0
Episode:3 Score:32.0
Episode:4 Score:17.0
Episode:5 Score:34.0


# Understanding The Environment
https://github.com/openai/gym/blob/master/gym/envs/classic_control/cartpole.py

In [ ]:
# 0-push cart to left, 1-push cart to the right
env.action_space.sample()

In [ ]:
# [cart position, cart velocity, pole angle, pole angular velocity]
env.observation_space.sample()

# 3. Train an RL Model

In [75]:
# Make your directories first
log_path = os.path.join('Training', 'Logs')

In [76]:
env = gym.make(environment_name)
env = DummyVecEnv([lambda: env])
model = PPO('MlpPolicy', env, verbose = 1, tensorboard_log=log_path)
# PPO??

Using cpu device


In [77]:
model.learn(total_timesteps=20000)

Logging to Training/Logs/PPO_1
-----------------------------
| time/              |      |
|    fps             | 1526 |
|    iterations      | 1    |
|    time_elapsed    | 1    |
|    total_timesteps | 2048 |
-----------------------------
-----------------------------------------
| time/                   |             |
|    fps                  | 1084        |
|    iterations           | 2           |
|    time_elapsed         | 3           |
|    total_timesteps      | 4096        |
| train/                  |             |
|    approx_kl            | 0.008796491 |
|    clip_fraction        | 0.104       |
|    clip_range           | 0.2         |
|    entropy_loss         | -0.686      |
|    explained_variance   | -0.0123     |
|    learning_rate        | 0.0003      |
|    loss                 | 6.83        |
|    n_updates            | 10          |
|    policy_gradient_loss | -0.0156     |
|    value_loss           | 54.5        |
-----------------------------------------
---

# 4. Save and Reload Model

In [78]:
PPO_path = os.path.join('Training', 'Saved Models', 'PPO_model')

In [79]:
model.save(PPO_path)

In [71]:
del model

In [72]:
model = PPO.load(PPO_path, env=env)

# 4. Evaluation

In [73]:
from stable_baselines3.common.evaluation import evaluate_policy

In [74]:
env = gym.make(environment_name,render_mode="human")
env = DummyVecEnv([lambda: env])
evaluate_policy(model, env, n_eval_episodes=10, render=True)

(500.0, 0.0)

In [35]:
env.close()

# 5. Test Model

In [80]:
env = gym.make(environment_name,render_mode="human")
env = DummyVecEnv([lambda: env])
obs = env.reset()
score = 0
while True:
    action, _states = model.predict(obs) #NOW USING MODEL HERE!!
    obs, rewards, done, info = env.step(action)
    env.render()
    score += reward
    if done: 
        print('info', info)
        print('reward', score)
        break
        

info [{'TimeLimit.truncated': True, 'terminal_observation': array([-0.00969358,  0.36210534,  0.10621299, -0.40715772], dtype=float32)}]
reward 500.0


In [113]:
env.close()

### Reviewing the model methods

In [53]:
# model method api
model.predict??

In [55]:
env = gym.make(environment_name)
env = DummyVecEnv([lambda: env])
obs = env.reset()

In [56]:
obs

array([[ 0.03617643, -0.01765439,  0.03399159, -0.02785905]],
      dtype=float32)

In [57]:
action, _states = model.predict(obs)

In [60]:
env.step(action)

(array([[ 0.03582334,  0.17696403,  0.03343441, -0.30962643]],
       dtype=float32),
 array([1.], dtype=float32),
 array([False]),
 [{'TimeLimit.truncated': False}])

# 6. Viewing Logs in Tensorboard

In [82]:
training_log_path = os.path.join('Training', 'Logs', 'PPO_1')
# training_log_path

In [84]:
!tensorboard --logdir={training_log_path}

TensorFlow installation not found - running with reduced feature set.
/home/alan/.local/lib/python3.8/site-packages/tensorboard_data_server/bin/server: /lib/x86_64-linux-gnu/libc.so.6: version `GLIBC_2.33' not found (required by /home/alan/.local/lib/python3.8/site-packages/tensorboard_data_server/bin/server)
/home/alan/.local/lib/python3.8/site-packages/tensorboard_data_server/bin/server: /lib/x86_64-linux-gnu/libc.so.6: version `GLIBC_2.34' not found (required by /home/alan/.local/lib/python3.8/site-packages/tensorboard_data_server/bin/server)
/home/alan/.local/lib/python3.8/site-packages/tensorboard_data_server/bin/server: /lib/x86_64-linux-gnu/libc.so.6: version `GLIBC_2.32' not found (required by /home/alan/.local/lib/python3.8/site-packages/tensorboard_data_server/bin/server)
Serving TensorBoard on localhost; to expose to the network, use a proxy or pass --bind_all
TensorBoard 2.14.0 at http://localhost:6006/ (Press CTRL+C to quit)
^C


# 7. Adding a callback to the training Stage

In [86]:
from stable_baselines3.common.callbacks import EvalCallback, StopTrainingOnRewardThreshold
import os

In [101]:
save_path = os.path.join('Training', 'Saved Models')
log_path = os.path.join('Training', 'Logs')

In [102]:
env = gym.make(environment_name) #,render_mode="human")
env = DummyVecEnv([lambda: env])

In [1]:
# StopTrainingOnRewardThreshold??

In [103]:
stop_callback = StopTrainingOnRewardThreshold(reward_threshold=190, verbose=1)
eval_callback = EvalCallback(env, 
                             callback_on_new_best=stop_callback, 
                             eval_freq=10000, 
                             best_model_save_path=save_path, 
                             verbose=1)

In [104]:
model = PPO('MlpPolicy', env, verbose = 1, tensorboard_log=log_path)

Using cpu device


In [105]:
model.learn(total_timesteps=20000, callback=eval_callback)

Logging to Training/Logs/PPO_5
-----------------------------
| time/              |      |
|    fps             | 1404 |
|    iterations      | 1    |
|    time_elapsed    | 1    |
|    total_timesteps | 2048 |
-----------------------------
-----------------------------------------
| time/                   |             |
|    fps                  | 1051        |
|    iterations           | 2           |
|    time_elapsed         | 3           |
|    total_timesteps      | 4096        |
| train/                  |             |
|    approx_kl            | 0.009287951 |
|    clip_fraction        | 0.0954      |
|    clip_range           | 0.2         |
|    entropy_loss         | -0.686      |
|    explained_variance   | 0.00997     |
|    learning_rate        | 0.0003      |
|    loss                 | 8.98        |
|    n_updates            | 10          |
|    policy_gradient_loss | -0.0144     |
|    value_loss           | 49.2        |
-----------------------------------------
---

In [106]:
model_path = os.path.join('Training', 'Saved Models', 'best_model')
model = PPO.load(model_path, env=env)

In [110]:
evaluate_policy(model, env, n_eval_episodes=20, render=False)
# evaluate_policy??

(360.65, 124.40027130195494)

In [111]:
env.close()

# 8. Changing Policies

In [ ]:
net_arch=[dict(pi=[128, 128, 128, 128], vf=[128, 128, 128, 128])]

In [ ]:
model = PPO('MlpPolicy', env, verbose = 1, policy_kwargs={'net_arch': net_arch})

In [ ]:
model.learn(total_timesteps=20000, callback=eval_callback)

# 9. Using an Alternate Algorithm

In [ ]:
from stable_baselines3 import DQN

In [ ]:
model = DQN('MlpPolicy', env, verbose = 1, tensorboard_log=log_path)

In [ ]:
model.learn(total_timesteps=20000, callback=eval_callback)

In [ ]:
dqn_path = os.path.join('Training', 'Saved Models', 'DQN_model')

In [ ]:
model.save(dqn_path)

In [ ]:
model = DQN.load(dqn_path, env=env)

In [ ]:
evaluate_policy(model, env, n_eval_episodes=10, render=True)

In [ ]:
env.close()